In [3200]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3201]:
from utils.UpdateDatabase import *

# Data Formatting

In [3202]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from urllib.parse import urljoin
from dotenv import load_dotenv
from datetime import *
from bson import ObjectId
import pandas as pd
import numpy as np
import requests
import random
import torch
import json
import os
load_dotenv()

True

In [3203]:
from utils.FlattenData import *
from modules.PremoAPI import *
from utils.SafeDataConverters import *

In [3204]:
api = PremoAPI()

In [3205]:
username = os.getenv("mongodb_user")
password = os.getenv("mongodb_password")

In [3206]:
from modules.database import Database
db = Database(username, password)

In [3207]:
play_data: list[dict] = api.play_data

In [3208]:
flat_out = flatten_output(db.to_json())
flat_in = flatten_input(db.to_json())

In [3209]:
df = pd.DataFrame(flat_out).drop(columns=['product_operation_id'], axis=1).sort_values(by=['order_id', 'sort_order'], ascending=True)
df['sort_order'] = df.groupby(['order_id', 'product_id'])['sort_order'].transform(lambda x: x.rank(method='dense').astype(int) - 1)
df = df.map(safe_to_datetime)
df = df.dropna()
df.head()

,order_id,delivery_date,order_created_at,product_id,material,color,task_title,task_duration,sort_order,workspace,start_at,end_at
2,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Vullen,3600,0,Soweco,2024-07-01 09:00:00,2024-07-01 10:00:00
3,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Tampon drukken (TB),1440,1,T2 1|2|3,2024-07-08 09:00:00,2024-07-08 09:24:00
4,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Inpakken,180,2,Inpak,2024-07-11 09:00:00,2024-07-11 09:03:00
5,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Expeditie,720,3,Expeditie,2024-07-12 09:00:00,2024-07-12 09:12:00
6,3,2024-07-24 00:00:00,2024-07-09 08:11:10,670cbb276c5384bfd1e19a62,Avant-knijper,13,Vullen,6120,0,Soweco,2024-07-10 09:00:00,2024-07-10 10:42:00


# Workspace Detection Model

In [3210]:
from modules.ColumnTransformer import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [3211]:
def preprocess_inputs(df):
    df = df.dropna()
    # Compute time differences in hours
    df['delivery_offset'] = (df['delivery_date'] - df['order_created_at']).dt.total_seconds() / 3600.0
    
    # Extract day of the week and time of day
    df['order_day_of_week'] = df['order_created_at'].dt.weekday  # 0 = Monday
    df['order_time_of_day'] = df['order_created_at'].dt.hour + df['order_created_at'].dt.minute / 60.0
    
    ct = ColumnTransformer(
        ['task_title', 'color', 'material', 'workspace'],
        ['delivery_offset', 'sort_order', 'task_duration', 'order_day_of_week', 'order_time_of_day'],
    )
    
    df = ct.fit_transform(df)
    
    # df['sort_order'] = df['sort_order']
    
    # Select features
    X_columns = ct.get_feature_names_out()
    X = df[X_columns]#.values

    return X

In [3212]:
def preprocess_outputs(df):
    scaler = StandardScaler()
    df = df.dropna()

    # Calculate the time difference between the order creation and the start of the order
    df['date_offset'] = (df['start_at'] - df['order_created_at']).dt.total_seconds()
    df['date_offset'] = df['date_offset'] / (3600 * 24) # Convert to days

    # Calculate the start time of the order
    df['start_time'] = df['start_at'].dt.hour + df['start_at'].dt.minute / 60.0

    # Normalize the values
    df[['date_offset', 'start_time']] = scaler.fit_transform(df[['date_offset', 'start_time']])

    # df['date_offset'] = (df['date_offset'] - df['date_offset'].mean()) / df['date_offset'].std()
    # df['start_time'] = (df['start_time'] - df['start_time'].mean()) / df['start_time'].std()
    
    # Select the columns to be used as output
    y = df[['date_offset', 'start_time']]# .values
    return y, scaler

In [3213]:
df

,order_id,delivery_date,order_created_at,product_id,material,color,task_title,task_duration,sort_order,workspace,start_at,end_at
2,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Vullen,3600,0,Soweco,2024-07-01 09:00:00,2024-07-01 10:00:00
3,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Tampon drukken (TB),1440,1,T2 1|2|3,2024-07-08 09:00:00,2024-07-08 09:24:00
4,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Inpakken,180,2,Inpak,2024-07-11 09:00:00,2024-07-11 09:03:00
5,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Expeditie,720,3,Expeditie,2024-07-12 09:00:00,2024-07-12 09:12:00
6,3,2024-07-24 00:00:00,2024-07-09 08:11:10,670cbb276c5384bfd1e19a62,Avant-knijper,13,Vullen,6120,0,Soweco,2024-07-10 09:00:00,2024-07-10 10:42:00
...,...,...,...,...,...,...,...,...,...,...,...,...
744,210,2024-10-03 00:00:00,2024-09-20 08:38:01,670cbb2f6c5384bfd1e19bb5,Elektronische aansteker HC,15,Zeefdrukken (ZT),9000,1,TX 1|2 MHM,2025-02-07 11:39:00,2025-02-07 14:09:00
745,210,2024-10-03 00:00:00,2024-09-20 08:38:01,670cbb2f6c5384bfd1e19bb5,Elektronische aansteker HC,15,Inpakken,900,3,Inpak,2025-02-12 11:39:10,2025-02-12 11:54:10
746,210,2024-10-03 00:00:00,2024-09-20 08:38:01,670cbb2f6c5384bfd1e19bb5,Elektronische aansteker HC,15,Expeditie,3600,4,Expeditie,2025-02-14 16:39:35,2025-02-14 17:39:35
1,367,2024-10-10 12:36:58,2024-10-11 12:36:58,670cbb166c5384bfd1e19a54,Promotierol mint dextrose 9,00,Inpakken,300,0,Inpak,2024-10-08 09:00:00,2024-10-08 09:05:00


In [3214]:
def add_random_rows(df, num_rows):
    current_rows = df.shape[0]
    for _ in range(num_rows):
        random_row = df.sample(n=1).copy()
        random_row['order_created_at'] = random_row['order_created_at'] + pd.to_timedelta(random.randint(1, 10), unit='d')
        random_row['start_at'] = random_row['order_created_at'] + pd.to_timedelta(random.randint(1, 10), unit='d')
        df = pd.concat([df, random_row], ignore_index=True)
    return df

# Add random rows to the input and output dataframes
num_random_rows = random.randint(15, 50)
df = add_random_rows(df, num_random_rows)
df = df.sort_values(by=['order_created_at', 'sort_order'], ascending=True)
df = df.groupby('order_id').apply(lambda x: x.sort_values(by='sort_order')).reset_index(drop=True)

/var/folders/4b/x6bk47k5499gbnw1_nzwq_7h0000gn/T/ipykernel_1751/2748705932.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('order_id').apply(lambda x: x.sort_values(by='sort_order')).reset_index(drop=True)


In [3215]:
X = preprocess_inputs(df)
y, yscaler = preprocess_outputs(df)
X.head()

,task_title_DP2,task_title_Expeditie,task_title_Inpakken,task_title_LD,task_title_Sample products,task_title_Tampon drukken (TA),task_title_Tampon drukken (TB),task_title_Tampon drukken (TM),task_title_Vullen,task_title_Zeefdrukken (ZA),...,workspace_Sample products,workspace_Soweco,workspace_T2 1|2|3,workspace_TX 1|2 MHM,workspace_Uitbesteden,delivery_offset,sort_order,task_duration,order_day_of_week,order_time_of_day
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.896704,0.00,0.011105,0.000000,0.942197
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.896704,0.25,0.004438,0.000000,0.942197
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.896704,0.50,0.000549,0.000000,0.942197
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.896704,0.75,0.002216,0.000000,0.942197
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.920094,0.00,0.018883,0.166667,0.215800


In [3216]:
y.head()

,date_offset,start_time
0,-1.251755,-1.24756
1,-1.080598,-1.24756
2,-1.007245,-1.24756
3,-0.982794,-1.24756
4,-1.049734,-1.24756


In [3217]:
from sklearn.model_selection import train_test_split

In [3218]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()

,task_title_DP2,task_title_Expeditie,task_title_Inpakken,task_title_LD,task_title_Sample products,task_title_Tampon drukken (TA),task_title_Tampon drukken (TB),task_title_Tampon drukken (TM),task_title_Vullen,task_title_Zeefdrukken (ZA),...,workspace_Sample products,workspace_Soweco,workspace_T2 1|2|3,workspace_TX 1|2 MHM,workspace_Uitbesteden,delivery_offset,sort_order,task_duration,order_day_of_week,order_time_of_day
344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.896704,0.00,0.166662,0.333333,0.620424
439,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.924873,0.50,0.001383,0.666667,0.289017
234,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.896704,0.00,0.000049,0.666667,0.938343
338,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.896704,0.50,0.000015,0.333333,0.412331
311,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.932171,0.25,0.022216,0.333333,0.250482


In [3219]:
from model import Model

In [3220]:
agent = Model()
agent.num_epochs = 1000
agent.batch_size = 16
agent.learning_rate = 0.001
agent.penalty_factor = 0.5
agent.model_path = os.path.join(os.getcwd(), 'models')

In [3221]:
model, history = agent.train(X_train, y_train, X_test, y_test)

Epoch 1/2000


/Users/grafistore/Desktop/GrafiStoreMLAPI/.conda/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9738 - val_loss: 0.9347
Epoch 2/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - loss: 0.8150 - val_loss: 0.8734
Epoch 3/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: 0.7217 - val_loss: 0.8194
Epoch 4/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - loss: 0.6374 - val_loss: 0.7462
Epoch 5/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - loss: 0.5364 - val_loss: 0.7355
Epoch 6/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - loss: 0.4981 - val_loss: 0.6637
Epoch 7/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - loss: 0.4277 - val_loss: 0.6375
Epoch 8/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - loss: 0.3827 - val_loss: 0.6418
Epoch 9/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - loss: 0.3236 - val_loss: 0.6201
Epoch 10/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 574us/step - loss: 0.2739 - val_loss: 0.6613
Epoch 11/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - loss: 0.2590 - val_loss: 0.6472
Epoch 12/2000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5

In [3222]:
loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss}")

Loss: 0.5828551650047302


### Get Prediction from play_data

In [3223]:
flat_play = flatten_input(play_data)
flat_play = pd.DataFrame(flat_play)

In [3224]:
flat_play = pd.DataFrame(flat_out).sort_values(by=['order_id', 'sort_order'], ascending=True)
flat_play['sort_order'] = flat_play.groupby(['order_id', 'product_id'])['sort_order'].transform(lambda x: x.rank(method='dense').astype(int) - 1)
flat_play = flat_play.map(safe_to_datetime)
flat_play = flat_play.drop(columns=['start_at', 'end_at'], axis=1)
flat_play = flat_play.dropna()
flat_play.head()

,order_id,delivery_date,order_created_at,product_id,material,color,product_operation_id,task_title,task_duration,sort_order,workspace
2,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,1,Vullen,3600,0,Soweco
3,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,2,Tampon drukken (TB),1440,1,T2 1|2|3
4,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,3,Inpakken,180,2,Inpak
5,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,4,Expeditie,720,3,Expeditie
6,3,2024-07-24 00:00:00,2024-07-09 08:11:10,670cbb276c5384bfd1e19a62,Avant-knijper,13,5,Vullen,6120,0,Soweco


In [3225]:
preprocessed_play = preprocess_inputs(flat_play)
preprocessed_play.head()

,task_title_DP2,task_title_Expeditie,task_title_Inpakken,task_title_LD,task_title_Sample products,task_title_Tampon drukken (TA),task_title_Tampon drukken (TB),task_title_Tampon drukken (TM),task_title_Vullen,task_title_Zeefdrukken (ZA),...,workspace_Sample products,workspace_Soweco,workspace_T2 1|2|3,workspace_TX 1|2 MHM,workspace_Uitbesteden,delivery_offset,sort_order,task_duration,order_day_of_week,order_time_of_day
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.896704,0.00,0.011105,0.00,0.942197
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.896704,0.25,0.004438,0.00,0.942197
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.896704,0.50,0.000549,0.00,0.942197
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.896704,0.75,0.002216,0.00,0.942197
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.920094,0.00,0.018883,0.25,0.215800


In [3226]:
product_operation_id_list: list = flat_play['product_operation_id'].values
order_created_at_list: list = flat_play['order_created_at'].values
task_duration_list: list = flat_play['task_duration'].values

In [3227]:
pred = agent.predict(preprocessed_play, order_created_at_list, task_duration_list, product_operation_id_list, yscaler)
pred = pd.DataFrame(pred).merge(flat_play, on='product_operation_id', how='left')
pred.head()

,product_operation_id,task_duration_x,start_at,end_at,order_created_at_x,order_id,delivery_date,order_created_at_y,product_id,material,color,task_title,task_duration_y,sort_order,workspace
0,1,3600.0,2024-07-01 09:00:00,2024-07-01 10:00:00,2024-07-08 14:28:51,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Vullen,3600,0,Soweco
1,2,1440.0,2024-07-08 09:00:00,2024-07-08 09:24:00,2024-07-08 14:28:51,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Tampon drukken (TB),1440,1,T2 1|2|3
2,3,180.0,2024-07-19 10:09:00,2024-07-19 10:12:00,2024-07-08 14:28:51,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Inpakken,180,2,Inpak
3,4,720.0,2024-07-19 10:12:00,2024-07-19 10:24:00,2024-07-08 14:28:51,1,2024-07-13 14:28:51,2024-07-08 14:28:51,670cbb276c5384bfd1e19a60,Katoenen draagtas gekleurd,06,Expeditie,720,3,Expeditie
4,5,6120.0,2024-08-08 09:20:00,2024-08-08 11:02:00,2024-07-09 08:11:10,3,2024-07-24 00:00:00,2024-07-09 08:11:10,670cbb276c5384bfd1e19a62,Avant-knijper,13,Vullen,6120,0,Soweco


In [3229]:
with open("output.json", "w") as f:
    f.write(pred.to_json(orient='records', date_format='iso'))